In [ ]:
import os
print(os.getcwd())
os.chdir('/content/drive/ToyProject')
print(os.getcwd())

/content


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shared drives/ToyProject'

In [ ]:
!ls
import zipfile

# 업로드된 파일의 이름
file_name = "train_images_5_class.zip"

# 압축 해제할 경로를 /content/로 설정
output_dir = "/content/drive/MyDrive/ToyProject"

# 경로가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 압축 해제
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("파일이 압축 해제되었습니다.")

data.ipynb  train_images_5_class.zip
파일이 압축 해제되었습니다.


In [ ]:
# 이미지 전처리, 넘파이 배열 형식으로 저장
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms

# 원본 데이터셋 경로
input_root_dir = '/content/drive/MyDrive/ToyProject/train_images_5_class'

# 분할된 데이터셋을 저장할 경로
output_root_dir = '/content/drive/MyDrive/ToyProject/preprocessed_images'
train_dir = os.path.join(output_root_dir, 'train')
val_dir = os.path.join(output_root_dir, 'val')
test_dir = os.path.join(output_root_dir, 'test')

# 데이터셋 폴더가 없으면 생성
for dir_path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# 이미지 전처리 파이프라인 (EfficientNet에 맞게)
transform = transforms.Compose([
    transforms.Resize((300, 300)),  # 300x300 크기로 리사이즈
    transforms.ToTensor(),          # 이미지를 텐서로 변환
    transforms.Normalize(           # 정규화 (EfficientNet에 맞게)
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# 각 클래스별로 데이터를 분할
for class_name in os.listdir(input_root_dir):
    class_dir = os.path.join(input_root_dir, class_name)

    if not os.path.isdir(class_dir):
        continue

    # 모든 이미지 파일 리스트
    images = os.listdir(class_dir)

    # 8:1:1 비율로 데이터를 나눔
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

    # 각 데이터셋에 해당하는 디렉토리 경로 설정
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    # 디렉토리가 없으면 생성
    for dir_path in [train_class_dir, val_class_dir, test_class_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # 이미지 전처리 후 NumPy 배열로 저장
    for img_name in train_images:
        img_path = os.path.join(class_dir, img_name)
        output_img_path = os.path.join(train_class_dir, os.path.splitext(img_name)[0] + '.npy')
        with Image.open(img_path) as img:
            img_processed = transform(img)  # 이미지 전처리
            img_array = img_processed.numpy()  # 텐서를 NumPy 배열로 변환
            np.save(output_img_path, img_array)  # NumPy 배열로 저장

    for img_name in val_images:
        img_path = os.path.join(class_dir, img_name)
        output_img_path = os.path.join(val_class_dir, os.path.splitext(img_name)[0] + '.npy')
        with Image.open(img_path) as img:
            img_processed = transform(img)  # 이미지 전처리
            img_array = img_processed.numpy()  # 텐서를 NumPy 배열로 변환
            np.save(output_img_path, img_array)  # NumPy 배열로 저장

    for img_name in test_images:
        img_path = os.path.join(class_dir, img_name)
        output_img_path = os.path.join(test_class_dir, os.path.splitext(img_name)[0] + '.npy')
        with Image.open(img_path) as img:
            img_processed = transform(img)  # 이미지 전처리
            img_array = img_processed.numpy()  # 텐서를 NumPy 배열로 변환
            np.save(output_img_path, img_array)  # NumPy 배열로 저장

print("이미지가 8:1:1 비율로 나누어지고, 전처리된 후 NumPy 배열로 저장되었습니다.")

이미지가 8:1:1 비율로 나누어지고, 전처리된 후 NumPy 배열로 저장되었습니다.


In [ ]:
# 모델에 넣어보기
import torch
import numpy as np
from torchvision import models

# EfficientNet-B3 모델 로드 (사전 학습된 가중치 사용)
weights = models.EfficientNet_B3_Weights.DEFAULT
model = models.efficientnet_b3(weights=weights)
model.eval()  # 모델을 평가 모드로 설정

# 넘파이 배열로 저장된 전처리된 이미지 파일 경로
img_path = '/content/drive/MyDrive/ToyProject/preprocessed_images/test/alert/10131474585_997d88e066_b.npy'  # .npy 파일 경로

# 넘파이 배열 로드
img_array = np.load(img_path)

# NumPy 배열을 PyTorch 텐서로 변환
input_tensor = torch.tensor(img_array, dtype=torch.float32)

# 배치 차원 추가 (모델 입력은 배치 단위로 해야 하므로)
input_batch = input_tensor.unsqueeze(0)

# GPU 사용 가능 시, GPU로 모델과 데이터 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_batch = input_batch.to(device)

# 모델에 입력하여 예측 수행
with torch.no_grad():  # 그래디언트 계산 비활성화 (평가 단계이므로)
    output = model(input_batch)

# 결과 해석 (가장 높은 확률의 클래스 추출)
_, predicted_idx = torch.max(output, 1)
print(f"Predicted class index: {predicted_idx.item()}")

In [ ]:
import json
import urllib

# ImageNet 클래스 인덱스 파일 다운로드
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
response = urllib.request.urlopen(url)
class_names = json.loads(response.read())

# 243번 클래스 이름 확인
predicted_class_name = class_names[243]
print(f"Predicted class name: {predicted_class_name}")

Predicted class name: Bullmastiff


In [ ]:
import shutil

# 압축할 폴더 경로
folder_path = '/content/drive/MyDrive/ToyProject/preprocessed_images'

# 압축 파일의 경로 (예: preprocessed_images.zip)
output_filename = '/content/drive/MyDrive/ToyProject/preprocessed_images.zip'

# 폴더를 압축
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_path)

'/content/drive/MyDrive/ToyProject/preprocessed_images.zip'

In [3]:
import tensorflow as tf
# TPU 설정 (옵션)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPUs를 검색 및 해결
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    print("No TPU found, using default strategy")
    strategy = tf.distribute.get_strategy()  # TPU가 없는 경우 기본 전략 사용 (예: CPU/GPU)

Running on TPU


In [1]:
from google.colab import drive
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Google 드라이브를 마운트
drive.mount('/content/drive')



# TPU 설정 (옵션)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPUs를 검색 및 해결
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    print("No TPU found, using default strategy")
    strategy = tf.distribute.get_strategy()  # TPU가 없는 경우 기본 전략 사용 (예: CPU/GPU)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running on TPU


In [2]:
# 데이터셋 로드 함수
def load_npy_file(file_path):
    return np.load(file_path)
def create_dataset_from_directory(directory, batch_size=32, shuffle=True):
    file_paths = []
    labels = []
    class_names = sorted(os.listdir(directory))

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.endswith('.npy'):
                file_paths.append(os.path.join(class_dir, file_name))
                labels.append(label)

    file_paths = np.array(file_paths)
    labels = np.array(labels)

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    def load_and_preprocess_image(file_path, label):
        image = tf.numpy_function(load_npy_file, [file_path], tf.float32)
        image = tf.reshape(image, [300, 300, 3])  # 원본 이미지의 예상 shape을 지정합니다.
        image = tf.image.resize(image, (224, 224))  # 이미지 크기 조정 (224x224)
        image = image / 255.0  # 정규화
        return image, label

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(file_paths))

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset, class_names  # 이 부분에서 dataset과 class_names를 반환합니다.


In [3]:
# 데이터 경로 설정
train_dir = '/content/drive/MyDrive/ToyProject/preprocessed_images_b3/train'
val_dir = '/content/drive/MyDrive/ToyProject/preprocessed_images_b3/val'

In [4]:
# 데이터셋 생성
train_dataset, class_names = create_dataset_from_directory(train_dir, batch_size=256, shuffle=True)

# 모델 정의 및 학습
with tf.distribute.get_strategy().scope():
    model = Sequential([
        EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='max'),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.3),  # Dropout 비율 낮춤
        Dense(len(class_names), activation='softmax')
    ])

    model.compile(optimizer=Adamax(learning_rate=0.0001),  # 학습률 낮춤
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # 콜백 설정
    early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='loss')  # monitor를 'loss'로 변경

    # 모델 학습
    history = model.fit(
        train_dataset,
        epochs=20,
        callbacks=[early_stopping, model_checkpoint]
    )


43941136/43941136 [==============================] - 0s 0us/step
Epoch 1/20
30/30 [==============================] - ETA: 0s - loss: 2.2424 - accuracy: 0.2173 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 795s 21s/step - loss: 2.2424 - accuracy: 0.2173
Epoch 2/20
30/30 [==============================] - 669s 21s/step - loss: 1.9898 - accuracy: 0.2481
Epoch 3/20
28/30 [===========================>..] - ETA: 44s - loss: 1.8543 - accuracy: 0.2783 

KeyboardInterrupt: 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import time

# Define possible hyperparameter values
batch_sizes = [16, 32, 64]
learning_rates = [0.0001, 0.001, 0.01]

# Define a dictionary to store results
results = {}

# Function to create and compile the model
def create_model(input_shape, learning_rate):
    model = Sequential([
        EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape, pooling='max'),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(len(class_names), activation='softmax')
    ])

    model.compile(optimizer=Adamax(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Function to perform grid search
def grid_search(train_dataset, input_shape):
    best_accuracy = 0
    best_params = {}

    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            print(f"Training with batch_size={batch_size} and learning_rate={learning_rate}")

            # Prepare the model
            model = create_model(input_shape, learning_rate)

            # Set callbacks
            early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
            model_checkpoint = ModelCheckpoint(f'best_model_bs{batch_size}_lr{learning_rate}.h5', save_best_only=True, monitor='loss')

            # Train the model
            start_time = time.time()
            history = model.fit(
                train_dataset,
                epochs=20,
                callbacks=[early_stopping, model_checkpoint],
                batch_size=batch_size,
                verbose=1
            )
            end_time = time.time()
            duration = end_time - start_time

            # Get the best accuracy from training
            max_accuracy = max(history.history['accuracy'])

            # Store results
            results[(batch_size, learning_rate)] = {
                'accuracy': max_accuracy,
                'duration': duration,
                'model_file': f'best_model_bs{batch_size}_lr{learning_rate}.h5'
            }

            # Update best parameters
            if max_accuracy > best_accuracy:
                best_accuracy = max_accuracy
                best_params = {'batch_size': batch_size, 'learning_rate': learning_rate}

    print("Best Hyperparameters:")
    print(f"Batch Size: {best_params['batch_size']}, Learning Rate: {best_params['learning_rate']}")
    print(f"Best Accuracy: {best_accuracy}")

    return best_params, results

# Assume train_dataset is already created and input_shape is defined
input_shape = (224, 224, 3)
best_params, results = grid_search(train_dataset, input_shape)

# Optionally, you can save the results to a file for further analysis
import json
with open('grid_search_results.json', 'w') as f:
    json.dump(results, f, indent=4)


Training with batch_size=16 and learning_rate=0.0001
Epoch 1/20
30/30 [==============================] - 696s 21s/step - loss: 2.1386 - accuracy: 0.2290
Epoch 2/20
30/30 [==============================] - 668s 21s/step - loss: 1.9373 - accuracy: 0.2672
Epoch 3/20
30/30 [==============================] - 663s 21s/step - loss: 1.8587 - accuracy: 0.2799
Epoch 4/20
30/30 [==============================] - 666s 21s/step - loss: 1.7487 - accuracy: 0.3130
Epoch 5/20
30/30 [==============================] - 666s 21s/step - loss: 1.6979 - accuracy: 0.3225
Epoch 6/20
30/30 [==============================] - 669s 21s/step - loss: 1.7135 - accuracy: 0.3137
Epoch 7/20
30/30 [==============================] - 668s 21s/step - loss: 1.7054 - accuracy: 0.3225
Epoch 8/20
30/30 [==============================] - 669s 21s/step - loss: 1.6151 - accuracy: 0.3421
Epoch 9/20
30/30 [==============================] - 668s 21s/step - loss: 1.6044 - accuracy: 0.3454
Epoch 10/20
30/30 [============================